# Transformer Attention

- Attention Mechanism : 특정 Layer층에서 출력되는 매 시점마다 전체 문장을 다시 참조하여 출력하는 학습 형태 
- 유사도를 계산하여, 가장 유사도가 높은 문장의 벡터를 찾고, 다음 Layer 반영 
- 어텐션 매커니즘의 핵심 개념 : 
    - 키 (Key) : 비교 대상이 되는 단어들의 표현 
    - 값 (Value) : 현재 처리중인 문장에 대한 대상 표현 
    - 쿼리 (Query) : 질문과 응답간에 얻어진 유사도 값 
    
- 트랜스포머에서 사용되는 핵심 Attention 
    - Muti-Head Attetion : 인코더와 디코더의 어탠션을 "헤드"요소를 통해 병렬 
        - Encoder Self Attention (Masked Decoder Self Attention) : 모델이 하나의 시퀀스 내에서 단어들 사이의 관계를 파악 
        - Encoder-Decoder Attention : 모델이 인코더의 출력과 디코더의 입력 사이의 관계를 파악 
    
    ![image2](https://wikidocs.net/images/page/31379/transformer_attention_overview.PNG)

In [1]:
import tensorflow as tf 
import re 

In [2]:
# 질의응답 데이터를 포지셔널 인코딩을 통해 위치정보가 포함된 텐서 데이터 셋 
save_path = '/chatbot_dataset'
dataset3 = tf.data.experimental.load(save_path)

Instructions for updating:
Use `tf.data.Dataset.load(...)` instead.


In [3]:
dataset3

<_LoadDataset element_spec=({'inputs': TensorSpec(shape=(None, 40), dtype=tf.int32, name=None), 'dec_inputs': TensorSpec(shape=(None, 39), dtype=tf.int32, name=None)}, {'outputs': TensorSpec(shape=(None, 39), dtype=tf.int32, name=None)})>

In [4]:
START_TOKEN = [8178] # <SOS> 
END_TOKEN = [8179]   # <EOS>
VOCAB_SIZE = 8180  # 단어가 하나의 정수로 매칭된 사전의 크기 
MAX_LENGTH = 40 # 하나의 문장이 정수의 토큰들로 표현되는 열의 크기  

**Chat Bot 구현**

In [5]:
import transformer_chatbot

In [6]:
tf.keras.backend.clear_session()

# 하이퍼파라미터를 지정 
D_MODEL = 256 # 트랜스 포머 내 모든 Layer안에 Node 수 
NUM_LAYERS = 2 # 인코더와 디코더 각각에 있는 Layer 수 
NUM_HEADS = 8 # 멀티-헤드 어텐션에서 사용되는 헤드의 수 
DFF = 512 # Feed Forward 신경망 내 Layer Node 수 
DROPOUT = 0.1 # 드롭아웃의 비율 

model = transformer_chatbot.transformer(vocab_size=VOCAB_SIZE, 
                                       num_layers=NUM_LAYERS,
                                       dff=DFF,
                                       d_model=D_MODEL,
                                       dropout=DROPOUT,
                                       num_heads=NUM_HEADS)

In [8]:
# 모델 컴파일을 위한 정확도 계산 함수를 생성 
# 문장의 최대 길이에서 시작 토큰과 종료 토큰을 제외하고 나머지 데이터를 이용해 
# 정확도를 계산 -> Weight Update 
def accuracy(y_true, y_pred):
    y_true = tf.reshape(y_true, shape=(-1, MAX_LENGTH-1))
    return tf.keras.metrics.sparse_categorical_accuracy(y_true, y_pred)

In [9]:
#  최적화 객체 생성  / 각 Node 수에 맞게 끔 Learning Rate 세팅 
learning_rate = transformer_chatbot.CustomSchedule(D_MODEL)

optimizer = tf.keras.optimizers.Adam(learning_rate, 
                                     beta_1=0.9,
                                     beta_2=0.98, epsilon=1e-9)
# adam : RMSprop 과 모멘텀 기법을 결합하여, 학습율도 조절하면서 모멘텀 벡터도 계산
# 각 최적화 함수의 이동평균(Moving Average)값을 이용해 Weight 최적화 
# beta : 모멘텀 및 스케일 조정 계수 

In [10]:
model.compile(optimizer=optimizer, 
              loss=transformer_chatbot.loss_function,
               metrics=[accuracy])

In [11]:
model.fit(dataset3, epochs=10) # 약 100회정도 학습 시, 대화 통하는 정도의 모델 

Epoch 1/10
185/185 ━━━━━━━━━━━━━━━━━━━━ 496s 2s/step - accuracy: 0.0199 - loss: 1.4217
Epoch 2/10
185/185 ━━━━━━━━━━━━━━━━━━━━ 531s 3s/step - accuracy: 0.0485 - loss: 1.1541
Epoch 3/10
185/185 ━━━━━━━━━━━━━━━━━━━━ 478s 3s/step - accuracy: 0.0493 - loss: 0.9600
Epoch 4/10
185/185 ━━━━━━━━━━━━━━━━━━━━ 468s 2s/step - accuracy: 0.0516 - loss: 0.8803
Epoch 5/10
185/185 ━━━━━━━━━━━━━━━━━━━━ 528s 3s/step - accuracy: 0.0546 - loss: 0.8264
Epoch 6/10
185/185 ━━━━━━━━━━━━━━━━━━━━ 487s 2s/step - accuracy: 0.0574 - loss: 0.7771
Epoch 7/10
185/185 ━━━━━━━━━━━━━━━━━━━━ 189s 755ms/step - accuracy: 0.0611 - loss: 0.7228
Epoch 8/10
185/185 ━━━━━━━━━━━━━━━━━━━━ 168s 907ms/step - accuracy: 0.0659 - loss: 0.6666
Epoch 9/10
185/185 ━━━━━━━━━━━━━━━━━━━━ 171s 920ms/step - accuracy: 0.0727 - loss: 0.6046
Epoch 10/10
185/185 ━━━━━━━━━━━━━━━━━━━━ 142s 765ms/step - accuracy: 0.0800 - loss: 0.5412


In [14]:
# 입력값 처리 
def preprocess_sentence(sentence):
    # 입력 문장에 특수기호 처리 
    sentence = re.sub(r"([?.!,])", r" \1 ", sentence)
    return sentence.strip()

In [17]:
def evaluate(sentence):
    # 입력 문장에 대한 전처리
    sentence = preprocess_sentence(sentence)

    # 입력 문장에 시작 토큰과 종료 토큰을 추가
    sentence = tf.expand_dims(
      START_TOKEN + tokenizer.encode(sentence) + END_TOKEN, axis=0)

    output = tf.expand_dims(START_TOKEN, 0)

    # 디코더의 예측 시작
    for i in range(MAX_LENGTH):
        predictions = model(inputs=[sentence, output], training=False)

        # 모델의 출력에서 마지막 단어를 선택해, 이를 바탕으로 다음 단어를 예측 
        # 문장내 단어를 하나씩 출력         
        predictions = predictions[:, -1:, :]
        # 예측된 단어를 변수로 선언 
        predicted_id = tf.cast(tf.argmax(predictions, axis=-1), tf.int32)

        # 만약 현재 시점의 예측 단어가 종료 토큰이라면 예측을 중단
        if tf.equal(predicted_id, END_TOKEN[0]):
              break

        # 현재 시점의 예측 단어를 output(출력)에 연결한다.
        # output은 for문의 다음 루프에서 디코더의 입력이 된다.
        output = tf.concat([output, predicted_id], axis=-1)

    # 단어 예측이 모두 끝났다면 output을 리턴.
    return tf.squeeze(output, axis=0)

In [23]:
def predict(sentence):
    prediction = evaluate(sentence)
    # 앞서 출력된 Matrix의 결과값을 하나씩 가져와, 단어사전에 있는 문자로 변환 
    predict_sentence = tokenizer.decode(
                            [i for i in prediction if i < tokenizer.vocab_size])
    print('입력 질문 : ',sentence)
    print('챗봇 대답 : ',predict_sentence)
    return predict_sentence

In [24]:
# 문자-숫자 사전(Voca) 불러오기
import pickle 

In [25]:
tokenizer = pickle.load(open('token.sav','rb'))

In [26]:
# 실제 질문 문장 입력 
input_sentance = input()
output = predict(input_sentance)
print(output)

이 과정이 끝나면 그녀에게 고백할꺼야!
입력 질문 :  이 과정이 끝나면 그녀에게 고백할꺼야!
챗봇 대답 :  직접 물어보세요 .
직접 물어보세요 .
